In [ ]:
!pip install git+https://github.com/huggingface/transformers accelerate qwen-vl-utils bitsandbytes

In [ ]:
from datasets import Dataset

# Percorso del file salvato
# json_path = "/kaggle/input/final-test-set/final_dataset.json"
dataset_path = '/kaggle/input/summarized-dataset/dataset_summarized'

#import shutil

#src = "/kaggle/input/new-correct-dataset/kaggle/working/new_correct_dataset"
#dst = "/kaggle/working/test_set_correct"

#shutil.copytree(src, dst, dirs_exist_ok=True)
#print(f"Copiato dataset da {src} a {dst}")

# Carica il dataset Hugging Face
# hf_dataset = Dataset.from_json(json_path)
hf_dataset = Dataset.load_from_disk(dataset_path)

# def update_image_k_path(example):
    #old_path = example['image_k']
    #suffix = old_path.split("new_data/")[-1]
    #new_path = f"/kaggle/input/new-dataset-images/new_data/{suffix}"
    #return {"image_k": new_path.replace("\\", "/")}

#hf_dataset = hf_dataset.map(update_image_k_path)
#hf_dataset.save_to_disk("/kaggle/working/new_correct_dataset_2")

# Controlla un esempio
print(hf_dataset)
print(hf_dataset['summarized_x_t'][850])

In [ ]:
!zip -r /kaggle/working/new_dataset_2.zip /kaggle/working/new_correct_dataset_2


In [ ]:
from transformers import pipeline

# Carica il classificatore zero-shot
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Etichette da prevedere
import re

LABELS = ["real", "fake"]

def extract_label_from_response(response: str) -> int:
    result = classifier(response, LABELS)
    return LABELS.index(result["labels"][0])

In [ ]:
import os
from PIL import Image
import torch
from transformers import (
    InstructBlipProcessor,
    InstructBlipForConditionalGeneration,
    GenerationConfig,
    AddedToken,
)
import bitsandbytes
import re

os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

FEW_SHOT_EXAMPLES = [
    {
        "context": "A busy street photo at midday with natural shadows, realistic reflections, and clear faces.",
        "label": "[Real]"
    },
    {
        "context": "A misty forest with glowing edges around trees and repeating texture patterns.",
        "label": "[Fake]"
    }
]


STATIC_PROMPTS = [
    "Is this image real or fake? Choose one: [Real] / [Fake].",
    "Analyze and classify: [Real] / [Fake].",
    "Based on lighting, texture, and edges, decide: [Real] / [Fake]."
]

class BlipTester:
    def __init__(self):
        self.device   = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_id = "Mediocreatmybest/instructblip-flan-t5-xl_8bit_nf4"

        # 1) Carica il modello in 8bit
        self.model = InstructBlipForConditionalGeneration.from_pretrained(
            self.model_id,
            load_in_8bit=True,
            device_map={"": 0},
        )

        # 2) Carica il processor
        self.processor = InstructBlipProcessor.from_pretrained(self.model_id)

        # 3) Aggiorna tokenizer per il token <image>
        self.processor.num_query_tokens = self.model.config.num_query_tokens
        img_tok = AddedToken("<image>", normalized=False, special=True)
        self.processor.tokenizer.add_tokens([img_tok], special_tokens=True)
        self.model.resize_token_embeddings(len(self.processor.tokenizer), pad_to_multiple_of=64)
        self.model.config.image_token_index = len(self.processor.tokenizer) - 1

        # DEBUG
        print("Image token index:", self.model.config.image_token_index)


    def build_prompt(self, prompt_type: str, x_t: str = "") -> str:
        few_shot = "".join([
            f"Example:\nScene Description: {ex['context']}\nLabel: {ex['label']}\n\n"
            for ex in FEW_SHOT_EXAMPLES
        ])
    
        if prompt_type == "dynamic" and x_t:
            return (
                f"{few_shot}"
                f"Scene Description: {x_t.strip()}\n"
                "Question: Is this image [Real] or [Fake]?\n"
                "Answer:"
            )
        else:
            template = STATIC_PROMPTS[int(prompt_type.split('_')[-1]) if '_' in prompt_type else 0]
            return (
                f"{few_shot}"
                f"Question: {template}\nAnswer:"
            )


    def generate(self, image_path: str, prompt: str) -> str:
        # Carica e ridimensiona
        img = Image.open(image_path).convert("RGB")

        # Prepara input (processor espande <image> internamente)
        inputs = self.processor(text=prompt, images=img, truncation=True, max_length=512, padding="max_length", return_tensors="pt").to(self.device)

        # Genera con sampling
        outputs = self.model.generate(
            **inputs,
            do_sample=False,  # più deterministico
            max_new_tokens=10, 
            eos_token_id=self.processor.tokenizer.eos_token_id,
        )

        # Decodifica
        return self.processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()

    def extract_label(self, response: str) -> int:
        return extract_label_from_response(response)

    def test(self, example: dict, prompt_type: str = "dynamic"):
        prompt   = self.build_prompt(prompt_type, example.get("summarized_x_t", ""))
        response = self.generate(example["image_k"], prompt)
        label    = self.extract_label(response)
        return label, response

# Wrapper globale
_tester_instance = BlipTester()
def test_blip2(example, prompt_type="static_0"):
    torch.cuda.empty_cache()
    return _tester_instance.test(example, prompt_type)


In [ ]:
from datasets import load_from_disk
from tqdm import tqdm
import pandas as pd

def run_test():
    #dataset = load_from_disk(DATASET_PATH)
    #for i in range(10):
    #print("Dataset structure:\n", hf_dataset)
    example = hf_dataset[1]
    print("Path:\n", example['image_k'])
    #print("Image:\n", example['image'])
    #print("Prompt:\n", example['x_t'])
    label, response = test_blip2(example, "static_0")
    print("Output label:\n", label)
    print("Output response:\n", response)

#run_test()


In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from datasets import load_from_disk

# DATASET_PATH   = "/kaggle/input/test-dataset-kaggle/test_dataset_kaggle"
CHECKPOINT_CSV = "/kaggle/working/dynamic_new_dataset_sum_InstructBlip.csv"
PROMPT_TYPE    = "dynamic"
CHECKPOINT_EVERY = 30  # salva ogni 30 campioni

# 1) Carica dataset
#dataset = load_from_disk(DATASET_PATH)

# 2) Se esiste già un CSV di checkpoint, riloadalo e salta i campioni già processati
if os.path.exists(CHECKPOINT_CSV):
    df = pd.read_csv(CHECKPOINT_CSV)
    processed_ids = set(df["img_id"])
else:
    df = pd.DataFrame(columns=["img_id","gt_label","pred_label","response"])
    processed_ids = set()

# 3) Loop con tqdm e checkpoint ogni N campioni
for sample in tqdm(hf_dataset, desc=f"Eval {PROMPT_TYPE}", dynamic_ncols=True):
    img_id = sample["img_id"]
    if img_id in processed_ids:
        continue

    try:
        pred_label, resp = test_blip2(sample, prompt_type=PROMPT_TYPE)
    except Exception as e:
        print("Errore nella generazione")
        pred_label, resp = -1, f"[ERROR] {e}"

    # prepara la nuova riga
    row = {
        "img_id":     img_id,
        "gt_label":   sample["label"],
        "pred_label": pred_label,
        "response":   resp
    }
    # concatena in un colpo solo
    df = pd.concat([df, pd.DataFrame([row])], ignore_index=True)
    processed_ids.add(img_id)

    # salva checkpoint
    if len(processed_ids) % CHECKPOINT_EVERY == 0:
        df.to_csv(CHECKPOINT_CSV, index=False)

# 4) Alla fine salva il CSV definitivo
df.to_csv(CHECKPOINT_CSV, index=False)
print("✅ Checkpoint salvato in:", CHECKPOINT_CSV)

# 5) Calcolo accuracy su quelli validi
valid = df["pred_label"] != -1
acc = (df.loc[valid, "gt_label"] == df.loc[valid, "pred_label"]).mean()
print(f"Accuracy {PROMPT_TYPE}: {acc:.4f}")
